In [1]:
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

sectors_physics_and_astronomy={
'Education':'https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10009', 
"Aerospace & Defense":'https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10002',
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10026",
"Management and Consulting":"https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10006",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10015",
} 


for j in range(0, len(sectors_physics_and_astronomy.keys())):
    options = Options()
    a='user-data-dir=C:/Users/dell/AppData/Local/Google/Chrome/User Data/Profile 17'
    options.add_argument(a)
    # PATH = "/Users/yourPath/Desktop/chromedriver"
    driver = webdriver.Chrome(options=options)
    site = list(sectors_physics_and_astronomy.values())[j]
    job_desc_Math_teacher={}
    driver.get(site)
    time.sleep(20)
    initialScroll = 0
    finalScroll = 1000
    start = time.time()
    while True:
        driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
        initialScroll = finalScroll
        finalScroll += 1000
        time.sleep(3)
        end = time.time()
        if round(end - start) > 25:
            break
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    
    import re
    jobs=soup.find_all('li',{'class':re.compile("^react-job-listing")})
      
    print(jobs)
    images=[]
    salaries=[]
    companies=[]
    job_titles=[]
    locations=[]

    for pro in jobs[:20]:
        try:
            image=pro.find('img')['src']
            images.append(image)
        except Exception as e:
            print(f"images_error--> {e}")
            images.append("nan")
        try:
            salary=pro.find('span', {"data-test":"detailSalary"}).text.strip()
            salaries.append(salary)
        except Exception as e:
            print(f"salaries_error--> {e}")
            salaries.append("nan")
        try:
            company=pro.find('a', {"class":"css-l2wjgv e1n63ojh0 jobLink"}).text.strip()
            companies.append(company)
        except Exception as e:
            print(f"companies_error--> {e}")
            companies.append("nan")
        try:
            job=pro.find('a', {"class":"jobLink css-1rd3saf eigr9kq2"}).text.strip()
            job_titles.append(job)
        except Exception as e:
            print(f"job_titles_error--> {e}")
            job_titles.append("nan")
        try:
            loc=pro.find('span', {"data-test":"emp-location"}).text.strip()
            locations.append(loc)
        except Exception as e:
            print(f"locations_error--> {e}")
            locations.append("nan")

    try:                                
        for i in range(1, len(salaries)+1):
            job_desc_butt=driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div[2]/section/article/div[1]/ul/li['+str(i)+']')
            job_desc_butt.click()
            time.sleep(7)
            src = driver.page_source
            soup = BeautifulSoup(src, 'lxml')
            job_descript=soup.find('div', {'class':'jobDescriptionContent desc'})
            job_description=job_descript.get_text()
            job_desc_Math_teacher[i]=job_description
    except Exception as e:
        print(e)
    driver.close()
  
    import pandas as pd
    print("creating df")
    df_teache=pd.DataFrame({
        "job_titles":job_titles,
        "job_description":list(job_desc_Math_teacher.values()),
        "images":images,
        "locations":locations,
        "salaries":salaries,
        "companies":companies,
        }
    )
    df_teache.to_csv(f"datas/datas_with_study/Aardewetenschappen_{list(sectors_physics_and_astronomy.keys())[j]}_last.csv", sep=',', index=False)

[<li class="react-job-listing css-1tmvqu eigr9kq3" data-ad-order-id="1136043" data-adv-type="GENERAL" data-brandviews="BRAND:n=jsearch-job-listing:eid=140437:jlid=1008504618710" data-id="1008504618710" data-is-easy-apply="false" data-is-organic-job="true" data-job-loc="Amsterdam" data-job-loc-id="3064478" data-job-loc-type="C" data-normalize-job-title="Assistant Professor" data-selected="true" data-sgoc-id="1006" data-test="jobListing" data-triggered-brandview="" style=""><div class="d-flex flex-column css-x75kgh e1rrn5ka3"><a class="jobLink" href="/partner/jobListing.htm?pos=101&amp;ao=1136043&amp;s=58&amp;guid=0000018746687c698bad551e9c5d5685&amp;src=GD_JOB_AD&amp;t=SR&amp;vt=w&amp;cs=1_2f8cf40f&amp;cb=1680513465741&amp;jobListingId=1008504618710&amp;jrtk=3-0-1gt36gv472hei001-1gt36gv4pg2qj800-8eb6a69fd2152d3d-" rel="nofollow noopener noreferrer" target="_blank" title="Vrije Universiteit Amsterdam"><span class="css-9ujsbx euyrj9o1" data-test="emp-logo"><img alt="Vrije Universiteit Ams

In [63]:
studies="""Actuarial Science
    Actuariële Wetenschappen
    Ancient studies
    Arabische taal en cultuur
    Archeologie
    Biomedische wetenschappen
    Business administration (bedrijfskunde)
    Business Analytics
    Beta-gamma (Natural and Social Sciences)
    Cognition, Language and Communication
    Communicatiewetenschap
    Communication Social Science
    Computational Social Science (Data- en sociale wetenschap)
    Cultural anthropology and Development Sociology
    Culturele antropologie en ontwikkelingssociologie 
    Cultuurwetenschappen
    Duitslandstudies
    Economics and Business Economics
    Englisch language and Culture
    European Studies
    Fiscaal recht
    Fiscale Economie
    Franse taal en cultuur
    Future Planet Studies
    Geneeskunde
    Geschiedenis
    Global Arts, Culture and Politics 
    Griekse en Latijnse taal en cultuur
    Hebreeuwse taal en cultuur
    Informatica
    Informatiekunde
    Interdisciplinaire sociale wetenschap
    Italië studies
    Kunstgeschiedenis
    Kunstmatige intelligentie
    Liberal Arts and Science
    Linguistics
    Literary and Cultural Analysis (literary studies)
    Media and Culture
    Media and Information
    Media en cultuur
    Medische informatiekunde
    Muziekwetenschap
    Natuur- en Sterrenkunde
    Nederlandse taal en cultuur
    Nieuwgriekse taal en cultuur
    Onderwijswetenschappen
    Oudheidwetenschappen
    Pedagogische wetenschappen
    Political Science
    Politicologie
    Politics, Psychology, Law and Economics (PPLE)
    Psychobiologie
    Psychology
    Rechtsgeleerdheid
    Religiewetenschappen
    Russische en Slavische studies
    Scandinavië studies
    Scheikunde
    Sign Language Linguistics (Linguistics)
    Sociale geografie en Planologie
    Sociology
    Spaanse en Latijns-Amerikaanse studies
    Tandheelkunde
    Theaterwetenschap
    Universitaire Pabo van Amsterdam
    Wiskunde
    Duitse Taal en Cultuur
    Literatuurwetenschap 
    Taalwetenschap
    Econometrie
    Artificial Intelligence
    Aarde, Economie en Duurzaamheid
    Aardewetenschappen
    Ancient studies
    Archeologie
    Bedrijfskunde
    Bestuurs- en organistatiewetenschap
    Bewegingswetenschap
    Business analytics
    Communicatie- en informatiewetenschappen
    communicatiewetenschap
    Communication and Information Studies
    Computer Science
    Criminologie
    Cultural Anthropology and Development Sociology
    Culturele antropologie en Ontwikkelingssociolgie
    Econometrics and Operations Research: Econometrics and Data Science
    Econometrie en Operationele Research
    Econometrics and Business economics
    Economie en Bedrijfskunde
    English Language and Communication Studies
    Farmaceutische Wetenschappen
    Filosofie
    Geneeskunde 
    Geschiedenis
    Gezondheidswetenschappen
    Griekse en Latijnse Taal en Cultuur
    History and International Studies
    International Business Administration
    Language and Media 
    Language Learning and Language Teaching
    Law in Society
    Literature and Society
    Mathematics (wiskunde)
    Mechanical Engineering
    Media en Journalistiek
    Media, Art, Design and Architecture
    Media, Kunst, Design en Architectuur
    Natuur- en sterrenkunde
    Notarieel Recht
    Oudheidwetenschappen
    Pedagogische wetenschappen
    Philosophy, Politics and Economics
    Political Science and Economics
    Politicologie
    scheikunde
    tandheelkunde
    Theology
    Universitaire Pabo"""
list_of_studies=studies.split("\n")


In [64]:
len(list_of_studies)

120

In [47]:
len(list(job_desc_Math_teacher.values()))

7

In [48]:
len(salaries)

20

In [33]:
for comp in companies:   
    driver.get(site1)
    time.sleep(20)                         
    comp_input=driver.find_element(By.XPATH, '//*[@id="sc.keyword"] ')  
    time.sleep(7)
    comp_input.click()
    comp_input.clear()
    comp_input.send_keys(comp)
    time.sleep(7)
    try:
        search_button=driver.find_element(By.XPATH, '//*[@id="scBar"]/div/button')
        search_button.click()
        time.sleep(7)

        first_comp=driver.find_element(By.XPATH, '//*[@id="Discover"]/div/div/div[1]/div[1]/div[1]/a[1]')
        first_comp.click()
        time.sleep(7)
                                                            #eiCell cell reviews 
        comp_review_button=driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div/div/div/div[1]/article[1]/div[1]/div[2]/div/div[2]/div/div[1]/div/a[1]')
        comp_review_button.click()
        time.sleep(7)
        
        initialScroll = 0                    
        finalScroll = 1000
        start = time.time()
        while True:
            driver.execute_script(f"window.scrollTo({initialScroll}, {finalScroll})")
            initialScroll = finalScroll
            finalScroll += 1000
            time.sleep(3)
            end = time.time()
            if round(end - start) > 20:
                break
        src = driver.page_source
        soup = BeautifulSoup(src, 'lxml')

        a =soup.find_all('p', {'class':"mt-0 mb-0 pb v2_EIReviewDetailsV2bodyColor v2EIReviewDetailsV2lineHeightLarge v2EIReviewDetailsV2_isCollapsed"})
        print("a is printing above")
        print("A IS PRINTING ABOVE")
        print(a)
        list_cons=[]
        list_pros=[]
        for i in a:
            try:
                list_cons.append(i.find('span', {'data-test':"cons"}).text.strip())
            except Exception as e:
                print(e)
        for i in a:
            # try:
            list_pros.append(i.find('span', {'data-test':"pros"}).text.strip())
            # except Exception as e:
                # print(e)
        if len(list_cons)==0:
            list_cons.append('NG')
        if len(list_pros)==0:
            list_pros.append('NG')

        pros.append(list_pros)
        cons.append(list_cons)
    except Exception as e:
        print(e)
        pros.append("NG")
        cons.append("NG")


KeyboardInterrupt: 

In [7]:
df_teache.locations

0    The Hague
1    The Hague
2    Amsterdam
3    Amsterdam
4    The Hague
5    The Hague
Name: locations, dtype: object

Done
Actuariële Wetenschappen={
'finanicial-services':'https://www.glassdoor.com/Job/netherlands--actuari%C3%ABle-wetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,37.htm?industryNId=10010', 
"Insurance":'https://www.glassdoor.com/Job/netherlands--actuari%C3%ABle-wetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,37.htm?industryNId=10014', 
"Information-Technology":'https://www.glassdoor.com/Job/netherlands--actuari%C3%ABle-wetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,37.htm?industryNId=10013'
}

Done
Ancient studies={
'finanicial-services':'https://www.glassdoor.com/Job/netherlands-studies-jobs-SRCH_IL.0,11_IN178_KO12,19.htm?industryNId=10010', 
"Education":'https://www.glassdoor.com/Job/netherlands-studies-jobs-SRCH_IL.0,11_IN178_KO12,19.htm?industryNId=10009', 
"Information-Technology":'https://www.glassdoor.com/Job/netherlands-studies-jobs-SRCH_IL.0,11_IN178_KO12,19.htm?industryNId=10013'
}

done 
 Business administration (bedrijfskunde)={
'finanicial-services':'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10010', 
"Pharmaceutical & Biotechnology":'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10005', 
"Information-Technology":'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10013'
Hotels-Travel-Accommodation':'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10025', 
"Management-Consulting":'https://www.glassdoor.com/Job/netherlands-business-administration-manager-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10006', 
}

rj
Sales Manager. ...
Business Consultant. ...
Financial Analyst. ...
Market Research Analyst. ...
Human Resources (HR) Specialist. ...
Loan Officer. ...
Meeting, Convention and Event Planner. ...
Business Consultant.

Done
business analytics={
'Education':'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10009', 
"Media-Communication":'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10016',
"Financial Services":'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10010', 
"Insurance":'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10014', 
"Energy-Mining-Utilities":'https://www.glassdoor.com/Job/netherlands-business-analytics-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10019',  
}

rj
Business Analytics Consultant
Senior Business Intelligence Analyst
Senior Business Analyst
Data Analysis 
Senior Data Analytics

done
Cognition, Language and Communication={
'Energy-Mining & Utilities':'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10019', 
"Financial-services":'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10010',
"Hotels-Travel-Accommodation":'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10025', 
"Pharmaceutical-Biotechnology":'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10005', 
"Human-Resources-Staffing":'https://www.glassdoor.com/Job/netherlands-language-and-communication-coach-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10026',  
}

rj

university instructor,
human factors consultant, 
industrial-organizational manager,
communication trainer,
communication consultant,
(online) content manager,
editor,
copywriter,
journalist
Business and marketing
Information technology
Health care
Law
Education
Research and development
Government and politics
Psychology

done

 Communicatiewetenschap={
'Education':'https://www.glassdoor.com/Job/netherlands--communicatiewetenschap-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?radius=0&industryNId=10009', 
"Human-Resources-Staffing":'https://www.glassdoor.com/Job/netherlands--communicatiewetenschap-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?radius=0&industryNId=10026',
"Media-Communication":'https://www.glassdoor.com/Job/netherlands--communicatiewetenschap-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?radius=0&industryNId=10016',  
}

rj

Public Relations Specialist
Marketing Manager
Social Media Manager
Advertising Account Executive
Corporate Communications Manager
Market Research Analyst
Media Planner
Event Planner
Content Strategist
Journalist


done
    Communication Social Science={
'Education':'https://www.glassdoor.com/Job/netherlands-social-science-research-assistant-jobs-SRCH_IL.0,11_IN178_KO12,45.htm?industryNId=10009', 
"Financial-Services":'https://www.glassdoor.com/Job/netherlands-social-science-research-assistant-jobs-SRCH_IL.0,11_IN178_KO12,45.htm?industryNId=10010',
"Healthcare":'https://www.glassdoor.com/Job/netherlands-social-science-research-assistant-jobs-SRCH_IL.0,11_IN178_KO12,45.htm?industryNId=10012', 
"Manufacturing":'https://www.glassdoor.com/Job/netherlands-social-science-research-assistant-jobs-SRCH_IL.0,11_IN178_KO12,45.htm?industryNId=10015',  
} 

rj

Public Relations Specialist
Marketing Coordinator
Advertising Manager
Social Media Specialist
Media Planner
Corporate Communications Manager
Event Planner
Content Writer
Grant Writer
Community Relations Specialist
Digital Content Strategist
Market Research Analyst
Journalist
Nonprofit Fundraiser
Speechwriter
Public Affairs Officer
Political Campaign Manager
Lobbyist
Customer Experience Specialist
Brand Manager

Done
computational socila science
rj

Data Scientist
Quantitative Researcher
Machine Learning Engineer
Research Analyst
Social Network Analyst
Computational Linguist
Information Scientist
Business Intelligence Analyst
Data Analyst
Market Researcher
UX Researcher
Statistician
Operations Research Analyst
Digital Marketing Analyst
Predictive Modeler
Risk Analyst
Data Mining Analyst
Big Data Engineer
Bioinformatics Scientist
Computational Economis

done
Culturele antropologie en ontwikkelingssociologie translation(Cultural anthropology and Development Sociology)=>
Cultural anthropology and Development Sociology={
'Education':'https://www.glassdoor.com/Job/amsterdam--cultural-anthropology-jobs-SRCH_IL.0,9_IC3064478_KO10,32.htm?industryNId=10009', 
"Construction-Repair-Maintenance-Services":'https://www.glassdoor.com/Job/amsterdam--cultural-anthropology-jobs-SRCH_IL.0,9_IC3064478_KO10,32.htm?industryNId=10007',
} 

rj
Anthropologist
Sociologist
Ethnographer
Social Researcher
Human Rights Officer
Cultural Advisor
Social Justice Advocate
Policy Analyst
Development Consultant
Gender Specialist
Community Development Officer
Nonprofit Program Manager
Public Health Researcher
International Aid Worker
Conflict Resolution Specialist
Humanitarian Relief Coordinator
Cultural Resource Manager
Community Organizer
Educator
Environmental Justice Advocate

Done

Englisch language and Culture={
'education':'https://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10009', 
"Hotels-Travel-Accommodation":'https://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10025',
"Retail-Wholesale":"https://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10022",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10015",
"human resources staffing":"https://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10026",
} 

rj
English Language Teacher/Instructor
Translator/Interpreter
Content Writer/Editor
Copywriter
Linguist
Language and Culture Consultant
Cultural Analyst
Cross-Cultural Trainer
Foreign Correspondent/Journalist
Language Program Director
Public Relations Specialist
Travel Writer/Blogger
Film/TV Critic
Speechwriter
International Sales Representative
Marketing Manager
Human Resources Specialist
Social Media Manager
Customer Service Representative
Event Planner.

done 

European Studies={
'education':'https://www.glassdoor.com/Job/netherlands--european-studies-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10009', 
"Aerospace-Defense":'https://www.glassdoor.com/Job/netherlands--european-studies-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10002',
"Energy Mining Utilities":"https://www.glassdoor.com/Job/netherlands--european-studies-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10019",
"Financial Services":"https://www.glassdoor.com/Job/netherlands--european-studies-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10010",
"Government  Public Administration":"https://www.glassdoor.com/Job/netherlands--european-studies-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10011",
} 

rj
Diplomat
International Development Specialist
International Relations Specialist
Policy Analyst
Researcher
Political Scientist
Historian
Journalist
Translator/Interpreter
Cultural Attaché
International Trade Specialist
International Business Manager
Public Affairs Specialist
Human Rights Advocate
Immigration Specialist
NGO Program Manager
European Union (EU) Policy Advisor
European Parliament Member/Assistant
Foreign Service Officer
European Economic and Financial Analyst.

Done
Fiscaal recht={
'education':'https://www.glassdoor.com/Job/netherlands--fiscaal-recht-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10009', 
"Healthcare":'https://www.glassdoor.com/Job/netherlands--fiscaal-recht-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10012',
"Information Technology":"https://www.glassdoor.com/Job/netherlands--fiscaal-recht-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10013",
"Financial Services":"https://www.glassdoor.com/Job/netherlands--fiscaal-recht-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10010",
"Government  Public Administration":"https://www.glassdoor.com/Job/netherlands--fiscaal-recht-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10011",
} 

rj
Tax lawyer
Tax advisor
Tax consultant
Tax manager
Tax inspector
Tax accountant
Corporate tax specialist
International tax specialist
Estate planning attorney
Trust and estate tax attorney
Tax court judge
Tax policy analyst
Tax examiner
Tax law professor
Transfer pricing specialist
Customs compliance specialist
VAT specialist
Tax compliance officer
Tax systems analyst
Tax software developer.

Done

    
Franse taal en cultuur={
'Human-Resources-Staffing':'https://www.glassdoor.com/Job/netherlands-franse-taal-en-cultuur-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10026', 
"Information Technology":'https://www.glassdoor.com/Job/netherlands-franse-taal-en-cultuur-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10013',
"Manufacturing":"https://www.glassdoor.com/Job/netherlands-franse-taal-en-cultuur-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10015",
"Retail  Wholesale":"https://www.glassdoor.com/Job/netherlands-franse-taal-en-cultuur-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10022",
"Management and Consulting":"https://www.glassdoor.com/Job/netherlands-franse-taal-en-cultuur-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10006",
} 

rj
French Language Teacher
Translator or Interpreter
French Tour Guide
International Sales Representative
Customer Service Representative for French-speaking clients
French Content Creator (Blogger, YouTuber, Podcaster)
French Localization Specialist
French Copywriter or Editor
French Social Media Manager
French Cultural Liaison Officer
French Business Development Manager
French Language Program Coordinator
French Diplomat or Foreign Service Officer
French Language Examiner or Evaluator
French Language Software Developer
French Language Researcher or Analyst
French Language Training Coordinator
French Language Consultant
French Language Voiceover Artist
French Language Marketing Manager


done
Future Planet Studies={
'Energy-Mining-Utilities':'https://www.glassdoor.com/Job/netherlands-future-planet-studies-jobs-SRCH_IL.0,11_IN178_KO12,33.htm?industryNId=10019', 
"Information Technology":'https://www.glassdoor.com/Job/netherlands-future-planet-studies-jobs-SRCH_IL.0,11_IN178_KO12,33.htm?industryNId=10013',
"Manufacturing":"https://www.glassdoor.com/Job/netherlands-future-planet-studies-jobs-SRCH_IL.0,11_IN178_KO12,33.htm?industryNId=10015",
"Pharmaceutical Biotechnology":"https://www.glassdoor.com/Job/netherlands-future-planet-studies-jobs-SRCH_IL.0,11_IN178_KO12,33.htm?industryNId=10005",
"Education":"https://www.glassdoor.com/Job/netherlands-future-planet-studies-jobs-SRCH_IL.0,11_IN178_KO12,33.htm?industryNId=10009",
} 

rj
Environmental Scientist
Climate Change Analyst
Sustainability Coordinator
Renewable Energy Engineer
Environmental Lawyer
Environmental Educator
Green Building Architect
Sustainable Agriculture Specialist
Environmental Economist
Conservation Biologist.
Environmental Policy Analyst
Water Resource Manager
Environmental Health and Safety Specialist
Carbon Footprint Analyst
Wildlife Biologist
Urban Planner (with an emphasis on sustainable development)
Environmental Communications Specialist
Ecotourism Coordinator
Green Business Manager
Natural Resource Manager

Done
Geneeskunde={
'Education':'https://www.glassdoor.com/Job/netherlands-medicine-jobs-SRCH_IL.0,11_IN178_KO12,20.htm?industryNId=10009', 
"Healthcare":'https://www.glassdoor.com/Job/netherlands-medicine-jobs-SRCH_IL.0,11_IN178_KO12,20.htm?industryNId=10012',
"Human Resources Staffing":"https://www.glassdoor.com/Job/netherlands-medicine-jobs-SRCH_IL.0,11_IN178_KO12,20.htm?industryNId=10026",
"Information Technology":"https://www.glassdoor.com/Job/netherlands-medicine-jobs-SRCH_IL.0,11_IN178_KO12,20.htm?industryNId=10013",
"Management Consulting":"https://www.glassdoor.com/Job/netherlands-medicine-jobs-SRCH_IL.0,11_IN178_KO12,20.htm?industryNId=10006",
} 

rj
General Practitioner/Family Medicine Physician
Surgeon
Obstetrician/Gynecologist
Pediatrician
Cardiologist
Oncologist
Anesthesiologist
Neurologist
Dermatologist
Psychiatrist
Radiologist
Pathologist
Emergency Medicine Physician
Endocrinologist
Infectious Disease Specialist
Geriatrician
Pulmonologist
Allergist/Immunologist
Ophthalmologist
Sports Medicine Physician


Done
    Geschiedenis={
'Healthcare':'https://www.glassdoor.com/Job/netherlands--geschiedenis-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10012', 
"Education":'https://www.glassdoor.com/Job/netherlands--geschiedenis-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10009',
"Human Resources Staffing":"https://www.glassdoor.com/Job/netherlands--geschiedenis-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10026",
"Information Technology":"https://www.glassdoor.com/Job/netherlands--geschiedenis-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10013",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands--geschiedenis-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10015",
} 

rj
Historian
Archivist
Curator
Museum Educator
Researcher
Writer/Author
Journalist
Teacher/Professor
Heritage Consultant
Historical Interpreter
Tour Guide
Preservation Specialist
Public Historian
Documentary Filmmaker
Genealogist
Cultural Resource Manager
Historical Society Administrator
Government Historian
Archaeologist
Film or TV Historical Consultant


Done
Global Arts, Culture and Politics ={
'Arts, Entertainment  Recreation':'https://www.glassdoor.com/Job/politics-jobs-SRCH_KO0,8.htm?industryNId=10004', 
"Construction, Repair  Maintenance Service":'https://www.glassdoor.com/Job/politics-jobs-SRCH_KO0,8.htm?industryNId=10007',
"Education":"https://www.glassdoor.com/Job/politics-jobs-SRCH_KO0,8.htm?industryNId=10009",
"Healthcare":"https://www.glassdoor.com/Job/politics-jobs-SRCH_KO0,8.htm?industryNId=10012",
"Information Technology":"https://www.glassdoor.com/Job/politics-jobs-SRCH_KO0,8.htm?industryNId=10013",
} 

rj

Cultural Affairs Officer
International Relations Specialist
Diplomat
Arts Program Manager
Museum Curator
Art Historian
Cultural Anthropologist
Public Policy Analyst
Nonprofit Program Director
International Development Consultant
Foreign Correspondent
Documentary Filmmaker
Cultural Heritage Manager
International Trade Specialist
Cross-Cultural Trainer
Global Marketing Manager
International Education Coordinator
International Human Rights Advocate
International Conflict Resolution Specialist
International Business Consultant

Done

    Griekse en Latijnse taal en cultuur={
'Education':'https://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10009', 
"Hotels Travel Accommodation":'https://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10025',
"Manufacturing":"hhttps://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10015",
"Retail  Wholesale":"https://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10022",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-language-and-culture-assistant-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10026",
} 

rj

Classicist
Latin Teacher
Greek Teacher
Museum Curator (of classical art and artifacts)
Archaeologist (specializing in Greek and Roman cultures)
Historical Fiction Writer
Translator (of ancient Greek and Latin texts)
Philology Professor
Mythology Researcher
Ancient History Professor
Epigraphy Specialist
Manuscript Librarian (for ancient texts)
Classics Book Editor
Classical Musician
Art Historian (with specialization in Greek and Roman art)
Latin Language Analyst (for language-based jobs such as translation, interpretation or speech writing)
Classical Archaeology Researcher
Academic Advisor (for Classics majors)
Classics Curriculum Developer
Lecturer in Ancient Languages and Cultures

Done 

Hebreeuwse taal en cultuur={
'Education':'https://www.glassdoor.com/Job/netherlands-language-and-culture-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10009', 
"Energy Mining & Utilities":'https://www.glassdoor.com/Job/netherlands-language-and-culture-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10019',
"Financial Services":"https://www.glassdoor.com/Job/netherlands-language-and-culture-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10010",
"Hotels and Travel Accommodation":"https://www.glassdoor.com/Job/netherlands-language-and-culture-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10025",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-language-and-culture-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10026",
} 

rj

Translator/Interpreter
Language Teacher
Cultural Consultant
Museum Curator
Historian
Researcher
Archivist
Linguist
Tour Guide
Non-profit Administrator
Community Outreach Coordinator
Intercultural Communication Specialist
Foreign Language Correspondent
Diplomat
Peace Corps Volunteer
Travel Writer
Cultural Event Planner
Human Rights Advocate
Media Relations Specialist
Social Media Manager

Done
   Informatica={
'Education':'https://www.glassdoor.com/Job/netherlands-informatica-jobs-SRCH_IL.0,11_IN178_KO12,23.htm?industryNId=10009', 
"Aerospace and Defense":'https://www.glassdoor.com/Job/netherlands-informatica-jobs-SRCH_IL.0,11_IN178_KO12,23.htm?industryNId=10002',
"Energy, Mining and Utilities":"https://www.glassdoor.com/Job/netherlands-informatica-jobs-SRCH_IL.0,11_IN178_KO12,23.htm?industryNId=10019",
"Financial Services":"https://www.glassdoor.com/Job/netherlands-informatica-jobs-SRCH_IL.0,11_IN178_KO12,23.htm?industryNId=10010",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-informatica-jobs-SRCH_IL.0,11_IN178_KO12,23.htm?industryNId=10026",
} 

rj

ETL Developer
Data Analyst
Data Architect
Business Intelligence Analyst
Informatica Administrator
Data Warehouse Developer
Data Engineer
Software Engineer
Integration Specialist
Technical Lead
Project Manager
Database Administrator
Solution Architect
Data Modeler
Data Scientist
Big Data Developer
Technical Consultant
Systems Analyst
Informatica Developer
Quality Assurance Engineer

Done
  Informatiekunde={
'Education':'https://www.glassdoor.com/Job/netherlands--informatiekunde-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10009', 
"Government and Public Administration":'https://www.glassdoor.com/Job/netherlands--informatiekunde-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10011',
"Information Technology":"https://www.glassdoor.com/Job/netherlands--informatiekunde-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10013",
"Financial Services":"https://www.glassdoor.com/Job/netherlands--informatiekunde-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10010",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands--informatiekunde-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10026",
} 

rj

Data Analyst
Business Analyst
Information Architect
Database Administrator
Systems Analyst
Data Scientist
IT Project Manager
Data Warehouse Specialist
IT Consultant
Technical Writer
Software Developer
Information Security Analyst
Business Intelligence Analyst
Technical Support Specialist
Network Administrator
Information Systems Manager
User Experience Designer
Web Developer
Cloud Engineer
Quality Assurance Tester

Done
Interdisciplinaire sociale wetenschap={
Education':'https://www.glassdoor.com/Job/netherlands-interdisciplinary-social-science-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10009', 
"Aerospace and Defense":'https://www.glassdoor.com/Job/netherlands-interdisciplinary-social-science-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10002',
"Financial Services":"https://wwhttps://www.glassdoor.com/Job/netherlands-interdisciplinary-social-science-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10010",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-interdisciplinary-social-science-jobs-SRCH_IL.0,11_IN178_KO12,44.htm?industryNId=10026",
}
rj
Social Researcher
Policy Analyst
Project Coordinator
Community Development Officer
Nonprofit Program Manager
Social Impact Analyst
Program Evaluation Specialist
Social Work Case Manager
Youth Outreach Worker
Health Educator
Human Services Coordinator
Community Outreach Coordinator
Grant Writer
Social Media Strategist
Diversity and Inclusion Specialist
Public Relations Specialist
Marketing Research Analyst
Fundraising Manager
Social Policy Advocate
International Development Coordinator

Done 
    Italië studies={
'Education':'https://www.glassdoor.com/Job/netherlands--italy-studies-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10009', 
"Information TechnologyAerospace and Defense":'https://www.glassdoor.com/Job/netherlands--italy-studies-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10013',
"Retail and Wholesale":"https://www.glassdoor.com/Job/netherlands--italy-studies-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10022",
"Pharmaceutical and Biotechnology":"https://www.glassdoor.com/Job/netherlands--italy-studies-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10005",
} 

rj
Italian Language Teacher
Tour Guide in Italy
Italian Culture Specialist
Italian History Researcher
Italian Cuisine Chef
Italian Fashion Designer
Italian Music Instructor
Italian Art Historian
Italian Wine Sommelier
Italian Translator
Italian Literature Professor
Italian Archaeologist
Italian Film Critic
Italian Travel Writer
Italian Business Consultant
Italian Immigration Lawyer
Italian Religious Studies Professor
Italian Environmental Researcher
Italian Sports Reporter
Italian Opera Singer.

Done
Kunstgeschiedenis={
'Education':'https://www.glassdoor.com/Job/netherlands-art-history-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10009', 
"Hotels  Travel Accommodation":'https://www.glassdoor.com/Job/netherlands-art-history-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10025',
"Information Technology":"https://www.glassdoor.com/Job/netherlands-art-history-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10013",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-art-history-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10026",
"Restaurants and Food Service":"https://www.glassdoor.com/Job/netherlands-art-history-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10021",
} 

rj
Art Historian
Museum Curator
Art Appraiser
Gallery Manager
Art Conservator
Art Auction Specialist
Art Researcher
Art Critic
Art Consultant
Art Educator
Art Cataloguer
Art Librarian
Art Writer
Art Authentication Specialist
Art Collections Manager
Art Registrar
Art Archivist
Art Historiographer
Art Documentation Specialist
Art Gallery Owner.

Done

Linguistics={
'Education':'https://www.glassdoor.com/Job/netherlands--linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10009', 
"Management and Consulting":'https://www.glassdoor.com/Job/netherlands--linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10006',
"Information Technology":"https://www.glassdoor.com/Job/netherlands--linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10013",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands--linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10015",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands--linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10026",
} 
rj
Linguistic Consultant
Language Analyst
Computational Linguist
Speech Language Pathologist
Language Teacher
Lexicographer
Bilingual Education Coordinator
Discourse Analyst
Translation Specialist
Language Technologist
Language Localization Manager
Forensic Linguist
Language Project Manager
Language Documentation Specialist
Applied Linguist
Language Assessment Specialist
Language Data Analyst
Language Policy Analyst
Speech Scientist
Language Access Coordinator


done 
Literary and Cultural Analysis (literary studies)={
'Education':'https://www.glassdoor.com/Job/netherlands-literary-studies-jobs-jobs-SRCH_IL.0,11_IN178_KO12,33.htm?industryNId=10009', 
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-literary-studies-jobs-jobs-SRCH_IL.0,11_IN178_KO12,33.htm?industryNId=10026",
} 
rj
Literary Editor
Literary Critic
Cultural Analyst
Literature Professor
Literary Agent
Copy Editor
Literary Translator
Researcher
Manuscript Editor
Freelance Writer
Writing Consultant
Content Manager
Creative Writing Instructor
Literary Publicist
Literary Reviewer
Literary Archivist
Literary Event Coordinator
Literary Grant Writer
Literary Research Assistant
Literary Magazine 

Done
Media-and-Culture={
'Education':'https://www.glassdoor.com/Job/netherlands--media-and-culture-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10009', 
"Aerospace and Defense":'https://www.glassdoor.com/Job/netherlands--media-and-culture-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10002',
"Financial Services":"https://www.glassdoor.com/Job/netherlands--media-and-culture-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10010",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands--media-and-culture-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10015",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands--media-and-culture-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10026",
} 

rj
Media Producer
Culture Writer
Media Analyst
Communications Specialist
Social Media Manager
Content Creator
Media Planner
Cultural Anthropologist
Media Relations Manager
Broadcast Journalist
Digital Media Strategist
Public Relations Specialist
Media Buyer
Culture Curator
Film Critic
Media Consultant
Cultural Heritage Manager
Multimedia Specialist
Brand Manager
Media Researcher

Done
   Media and Information={
'Education':'https://www.glassdoor.com/Job/netherlands--media-and-information-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10009', 
"Aerospace and Defense":'https://www.glassdoor.com/Job/netherlands--media-and-information-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10002',
"Information Technology":"https://www.glassdoor.com/Job/netherlands--media-and-information-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10013",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands--media-and-information-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10015",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands--media-and-information-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10026",
} 

rj
Information Analyst
Media Manager
Digital Content Strategist
Information Architect
Media Planner
Content Manager
Media Analyst
Information Specialist
Social Media Coordinator
Information Consultant
Digital Librarian
Media Relations Specialist
Information Security Analyst
Content Editor
Media Buyer
Information Scientist
User Experience Designer
Digital Asset Manager
Media Researcher
Information Officer

Done
 Medische informatiekunde={
'Education':'https://www.glassdoor.com/Job/netherlands--medische-informatiekunde-jobs-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10009', 
"Healthcare":'https://www.glassdoor.com/Job/netherlands--medische-informatiekunde-jobs-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10012',
"Information Technology":"https://www.glassdoor.com/Job/netherlands--medische-informatiekunde-jobs-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10013",
"Management and Consulting":"https://www.glassdoor.com/Job/netherlands--medische-informatiekunde-jobs-jobs-SRCH_IL.0,11_IN178_KO12,42.htm?industryNId=10006",

} 

rj
Medical Informatics Specialist
Health Information Manager
Clinical Informatics Analyst
Data Scientist
Healthcare Data Analyst
Clinical Data Coordinator
Health Information Technician
Healthcare Informatics Consultant
Medical Software Developer
Clinical Informatics Manager
Health Informatics Researcher
Electronic Medical Record (EMR) Implementation Specialist
Clinical Decision Support Analyst
Health Information Systems Administrator
Telehealth Solutions Specialist
Medical Informatics Educator
Health IT Project Manager
Healthcare Compliance Analyst
Population Health Analyst
Health Data Quality Analyst

Done
 Natuur- en Sterrenkunde={
'Education':'https://www.glassdoor.com/Job/physics-and-astronomy-jobs-SRCH_KO0,21.htm?industryNId=10009', 
"Aerospace and Defense":'https://www.glassdoor.com/Job/physics-and-astronomy-jobs-SRCH_KO0,21.htm?industryNId=10002',
"Information Technology":"https://www.glassdoor.com/Job/physics-and-astronomy-jobs-SRCH_KO0,21.htm?industryNId=10013",
"Energy and Mining & Utilities":"https://www.glassdoor.com/Job/physics-and-astronomy-jobs-SRCH_KO0,21.htm?industryNId=10019",

} 

rj
Astrophysicist
Research Scientist
Data Scientist
Physicist
Astronomy Software Engineer
Science Writer
Observatory Technician
Science Educator
Aerospace Engineer
Planetarium Director
Science Illustrator
Cosmologist
Astronomy Outreach Specialist
Science Museum Curator
Spacecraft Engineer
Astronomical Instrumentation Engineer
Meteorologist
Science Policy Advisor
Science Fiction Writer
Amateur Astronomer

Done

    Nederlandse taal en cultuur={
'Arts,Entertainment and Recreation':'https://www.glassdoor.com/Job/dutch-language-and-culture-jobs-jobs-SRCH_KO0,31.htm?industryNId=10004', 
"Restaurants and Food Service":'https://www.glassdoor.com/Job/dutch-language-and-culture-jobs-jobs-SRCH_KO0,31.htm?industryNId=10021',
"Information Technology":"https://www.glassdoor.com/Job/dutch-language-and-culture-jobs-jobs-SRCH_KO0,31.htm?industryNId=10013",
"Manufacturing":"https://www.glassdoor.com/Job/dutch-language-and-culture-jobs-jobs-SRCH_KO0,31.htm?industryNId=10015",
"Hotels and Travel Accommodation":"https://www.glassdoor.com/Job/dutch-language-and-culture-jobs-jobs-SRCH_KO0,31.htm?industryNId=10025",
} 

rj
Dutch Language Teacher
Copywriter
Translator
Editor
Journalist
Interpreter
Language Program Coordinator
Museum Curator
Cultural Heritage Manager
Tour Guide
Public Relations Specialist
Marketing Coordinator
Content Creator
Localization Specialist
Speech Therapist
Language Assessor
Linguistic Researcher
Archivist
Dutch Language Editor
Cultural Event Coordinator

Done

 Onderwijswetenschappen={
'Education':'https://www.glassdoor.com/Job/netherlands--onderwijswetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10009', 
"Insurance":'https://www.glassdoor.com/Job/netherlands--onderwijswetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10014',
"Information Technology":"https://www.glassdoor.com/Job/netherlands--onderwijswetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10013",
"Management and Consulting":"https://www.glassdoor.com/Job/netherlands--onderwijswetenschappen-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10006",

} 

rj
Docent Onderwijswetenschappen
Onderwijskundig adviseur
Projectleider onderwijsontwikkeling
Beleidsmedewerker onderwijs
Onderzoeker onderwijs
Onderwijsontwikkelaar
Kwaliteitsmedewerker onderwijs
Toetsdeskundige
Adviseur leertrajecten
Trainer/Coach onderwijs
Coördinator educatie
Programmacoördinator
Docent Pedagogiek
Onderwijsmanager
Studieadviseur
Examencoördinator
Onderwijsinspecteur
Loopbaancoach onderwijs
Begeleider studiekeuze
Begeleider studenten met functiebeperkingen

Done
Oudheidwetenschappen={
'Education':'https://www.glassdoor.com/Job/netherlands-ancient-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,33.htm?industryNId=10009', 
"Hotels and Travel Accommodation":"https://www.glassdoor.com/Job/netherlands-ancient-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,33.htm?industryNId=10025",
} 

rj
Museum curator
Archivist
Librarian
Historical researcher
Archaeologist
Teacher or professor
Academic advisor
Tour guide or travel writer
Art historian
Journalist or writer
Translator
Cultural heritage consultant
Preservationist
Philologist
Linguist
Historic preservation planner
Historic site manager
Editorial assistant
Exhibition designer
Academic librarian

Done

Pedagogische wetenschappen={
'Education':'https://www.glassdoor.com/Job/netherlands-pedagogische-wetenschappen-jobs-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10009', 
"Healthcare":'https://www.glassdoor.com/Job/netherlands-pedagogische-wetenschappen-jobs-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10012',
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-pedagogische-wetenschappen-jobs-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10026",
"Management and Consulting":"https://www.glassdoor.com/Job/netherlands-pedagogische-wetenschappen-jobs-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10006",
"Government and Public Administration":"https://www.glassdoor.com/Job/netherlands-pedagogische-wetenschappen-jobs-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10011",
} 

rj

Onderwijskundige 
Kindercoach 
Schoolpsycholoog 
Pedagogisch adviseur 
Docent of professor
Didactisch ontwikkelaar 
Beleidsadviseur onderwijs
Trainer of coach
Opleidingsadviseur
Onderwijskwaliteitsmedewerker 
Jeugdhulpverlener 
Jeugd- en gezinscoach 
Schoolbegeleider 
Opleidingscoördinator
Loopbaanadviseur
Pedagogisch medewerker 
Assessmentpsycholoog
Begeleider speciaal onderwijs
Studieadviseur 
Kennismanager onderwijs

Done
Political Science={
'Education':'https://www.glassdoor.com/Job/netherlands--political-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10009', 
"Energy, Mining and Utilities":'https://www.glassdoor.com/Job/netherlands--political-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10019',
"Financial Services":"https://www.glassdoor.com/Job/netherlands--political-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10010",
"Information Technology":"https://www.glassdoor.com/Job/netherlands--political-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10013",
"Pharmaceutical and Biotechnology":"https://www.glassdoor.com/Job/netherlands--political-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10005",
}

rj
Policy advisor
Political consultant
Research analyst
Public affairs consultant
International development specialist
Government affairs director
Human rights advocate
Campaign manager
Diplomat
International relations specialist
Lobbyist
Non-profit program director
Political journalist
Public opinion researcher
Strategic communications specialist
Social media manager
Public relations specialist
Foreign service officer
Political risk analyst
Civil service 

Done

 Politicologie={
'Education':'https://www.glassdoor.com/Job/netherlands--politicologie-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10009', 
"Construction, Repair and Maintenance Services":'https://www.glassdoor.com/Job/netherlands--politicologie-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10007',
"Financial Services":"https://www.glassdoor.com/Job/netherlands--politicologie-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10010",
"Information Technology":"https://www.glassdoor.com/Job/netherlands--political-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,35.htm?industryNId=10013",
"Government and Public Administration":"https://www.glassdoor.com/Job/netherlands--politicologie-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10011",
} 

rj
Political analyst
Public policy advisor
Government affairs director
Campaign strategist
Diplomat
Research analyst
Public opinion researcher
Political journalist
Communications specialist
Political consultant
Public affairs director
Non-profit program director
Lobbyist
Human rights advocate
International development specialist
Foreign service officer
Public relations specialist
Social media manager
Conflict resolution specialist
Academic researcher

Done

Politics={
'Education':'https://www.glassdoor.com/Job/netherlands-politics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10009', 
"Insurance":'https://www.glassdoor.com/Job/netherlands-politics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10014',
"Management and Consulting":"https://www.glassdoor.com/Job/netherlands-politics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10006",
"Information Technology":"https://www.glassdoor.com/Job/netherlands-politics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10013",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-politics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,25.htm?industryNId=10026",
} 

rj

Political analyst
Policy advisor
Government affairs director
Campaign strategist
Diplomat
Research analyst
Public opinion researcher
Political journalist
Communications specialist
Political consultant
Public affairs director
Non-profit program director
Lobbyist
Human rights advocate
International development specialist
Foreign service officer
Public relations specialist
Social media manager
Conflict resolution specialist
Academic researcher

Done
Psychology={
'Education':'https://www.glassdoor.com/Job/netherlands-psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,27.htm?industryNId=10009', 
"Government and Public Administration":'https://www.glassdoor.com/Job/netherlands-psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,27.htm?industryNId=10011',
"Healthcare":"https://www.glassdoor.com/Job/netherlands-psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,27.htm?industryNId=10012",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,27.htm?industryNId=10026",
"Information Technology":"https://www.glassdoor.com/Job/netherlands-psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,27.htm?industryNId=10013",
} 

rj
Clinical Psychologist
Research Psychologist
School Psychologist
Neuropsychologist
Industrial-Organizational Psychologist
Forensic Psychologist
Developmental Psychologist
Educational Psychologist
Social Psychologist
Health Psychologist
Consumer Psychologist
Counseling Psychologist
Experimental Psychologist
Rehabilitation Psychologist
Sports Psychologist
Cognitive Psychologist
Environmental Psychologist
Media Psychologist
Geriatric Psychologist
Child Psychologist

Done
Law and Economics={
'Education':'https://www.glassdoor.com/Job/netherlands-law-and-economics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10009', 
"Energy,Mining and Utilities":'https://www.glassdoor.com/Job/netherlands-law-and-economics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10019',
"Financial Services":"https://www.glassdoor.com/Job/netherlands-law-and-economics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10010",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-law-and-economics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10026",
"Information Technology":"https://www.glassdoor.com/Job/netherlands-law-and-economics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10013",
} 

rj
Corporate Counsel
Mergers and Acquisitions Lawyer
Tax Lawyer
Antitrust Lawyer
Securities Lawyer
Intellectual Property Lawyer
Contract Lawyer
Compliance Officer
Risk Manager
Arbitrator
Mediator
Economic Analyst
Financial Analyst
Investment Banker
Business Consultant
Data Analyst
Operations Manager
Human Resources Manager
Marketing Manager
Product Manager

Done
   Psychology={
'Education':'https://www.glassdoor.com/Job/netherlands--psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10009', 
"Government and Public Administration":'https://www.glassdoor.com/Job/netherlands--psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10011',
"Healthcare":"https://www.glassdoor.com/Job/netherlands--psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10012",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands--psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10026",
"Information Technology":"https://www.glassdoor.com/Job/netherlands--psychology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10013",
} 

rj
Psychologist
Clinical Psychologist
Educational Psychologist
Child Psychologist
Neuropsychologist
Forensic Psychologist
Health Psychologist
Organizational Psychologist
Developmental Psychologist
Occupational Psychologist
Geriatric Psychologist
Sports Psychologist
Social Psychologist
Positive Psychologist
Experimental Psychologist
Cognitive Psychologist
Clinical Neuropsychologist
Educational Neuroscientist
Evolutionary Psychologist
Personality Psychologist

Done 
  Rechtsgeleerdheid={
'Education':'https://www.glassdoor.com/Job/netherlands-rechtsgeleerdheid-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10009', 
"Management and Consulting":'https://www.glassdoor.com/Job/netherlands-rechtsgeleerdheid-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10006',
"Legal":"https://www.glassdoor.com/Job/netherlands-rechtsgeleerdheid-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10001",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-rechtsgeleerdheid-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10026",
"Information Technology":"https://www.glassdoor.com/Job/netherlands-rechtsgeleerdheid-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10013",
} 
rj
Lawyer
Legal Advisor
Legal Counsel
Legal Consultant
Corporate Lawyer
Tax Lawyer
Intellectual Property Lawyer
Human Rights Lawyer
Environmental Lawyer
Criminal Defense Lawyer
Employment Lawyer
Immigration Lawyer
Family Lawyer
Insurance Lawyer
Litigation Lawyer
Public Defender
Legal Analyst
Legal Researcher
Compliance Officer
Mediator


Done
 Religiewetenschappen={
'Education':'https://www.glassdoor.com/Job/netherlands-religious-studies-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10009', 
"Financial Services":'https://www.glassdoor.com/Job/netherlands-religious-studies-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10010',
"Government and Public Administration":"https://www.glassdoor.com/Job/netherlands-religious-studies-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10011",
"Government & Public Administration":"https://www.glassdoor.com/Job/netherlands-religious-studies-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10011",
"Information Technology":"https://www.glassdoor.com/Job/netherlands-religious-studies-jobs-jobs-SRCH_IL.0,11_IN178_KO12,34.htm?industryNId=10013",
} 

rj
Religious Studies Researcher
Professor of Religious Studies
Interfaith Coordinator
Chaplain
Youth Minister
Religious Educator
Church Administrator
Missionary
Pastoral Counselor
Spiritual Director
Nonprofit Director
Humanitarian Aid Worker
Conflict Resolution Specialist
Interfaith Dialogue Facilitator
Theology Teacher
Bible Translator
Religious Writer
Museum Curator
Religious Tour Guide
Religious Musician

Done
Scandinavië studies={
'Human Resources and Staffing':'https://www.glassdoor.com/Job/netherlands--scandinavi%C3%AB-studies-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10026', 
"Manufacturing":'https://www.glassdoor.com/Job/netherlands--scandinavi%C3%AB-studies-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10015',
"Transportation and Logistics":"https://www.glassdoor.com/Job/netherlands--scandinavi%C3%AB-studies-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10024"
} 

rj
Researcher in Scandinavian Studies
Professor of Scandinavian Studies
Translator of Scandinavian Languages
Language Instructor of Scandinavian Languages
Nordic Culture Specialist
Nordic Art Expert
Scandinavian History Scholar
Scandinavian Literature Critic
Nordic Film and TV Expert
Study Abroad Program Coordinator for Scandinavia
Cultural Exchange Program Coordinator for Scandinavia
Nordic Business Specialist
Scandinavian Tourism Expert
Import/Export Specialist for Scandinavian Products
Nordic Environmental Policy Expert
Nordic Political Analyst
Scandinavian Food and Beverage Specialist
Scandinavian Fashion and Design Expert
Nordic Music and Performing Arts Specialist
Scandinavian Marketing and Advertising Consultant

Done
Sign Language Linguistics (Linguistics)={
'Management and Consulting':'https://www.glassdoor.com/Job/netherlands-sign-language-jobs-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10006', 
"Manufacturing":'https://www.glassdoor.com/Job/netherlands-sign-language-jobs-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10015',
"Pharmaceutical and Biotechnology":"https://www.glassdoor.com/Job/netherlands-sign-language-jobs-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10005",
"Real Estate":"https://www.glassdoor.com/Job/netherlands-sign-language-jobs-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10020",
"Information Technology":"https://www.glassdoor.com/Job/netherlands-sign-language-jobs-jobs-SRCH_IL.0,11_IN178_KO12,30.htm?industryNId=10013",
} 

rj
Lecturer in Sign Language and Deaf Studies
Sign Language Interpreter
Researcher in Sign Linguistics
Sign Language Teacher
Communication Advisor for Deaf and Hard of Hearing Persons
Sign Language Coordinator
Sign Language Expert
Sign Language Tutor
Accessibility Coordinator for Deaf and Hard of Hearing Persons
Sign Language Consultant
Sign Language and Deaf Culture Specialist
Speech Therapist for Deaf and Hard of Hearing Persons
Sign Language Instructor
Disability Support Specialist
Sign Language Program Coordinator
Language and Communication Specialist for Deaf and Hard of Hearing Persons
Sign Language Expert for Media and Entertainment Industry
Sign Language Research Assistant
Sign Language Outreach Coordinator
Sign Language Technology Specialist

Done
sectors_physics_and_astronomy={
'Construction, Repair and Maintenance Service':'https://www.glassdoor.com/Job/netherlands-sociale-geografie-en-planologie-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10007', 
"Human Resources and Staffing":'https://www.glassdoor.com/Job/netherlands-sociale-geografie-en-planologie-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10026',
"Management & Consulting":"https://www.glassdoor.com/Job/netherlands-sociale-geografie-en-planologie-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10006",
"Real Estate":"https://www.glassdoor.com/Job/netherlands-sign-language-jobs-jobs-SRCHhttps://www.glassdoor.com/Job/netherlands-sociale-geografie-en-planologie-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10020",
"Government and Public Administration":"https://www.glassdoor.com/Job/netherlands-sign-language-jobs-jobs-SRCH_IL.0,11_IN178_KO12https://www.glassdoor.com/Job/netherlands-sociale-geografie-en-planologie-jobs-SRCH_IL.0,11_IN178_KO12,43.htm?industryNId=10011",
} 

rj

Stedenbouwkundig ontwerper
Beleidsadviseur Ruimtelijke Ordening
Projectleider Gebiedsontwikkeling
Adviseur Mobiliteit
Onderzoeker Sociale Geografie
Data-analist Ruimtelijke Ordening
Beleidsmedewerker Duurzaamheid
Gebiedsmanager
Stedelijk planner
Procesmanager Ruimtelijke Ordening
Omgevingsmanager
Adviseur Landschapsarchitectuur
Programmacoördinator Gebiedsontwikkeling
Consultant Vastgoedontwikkeling
Medewerker Gebiedsinrichting
Landschapsontwerper
Beleidsadviseur Wonen
Duurzaamheidsadviseur
Milieukundig adviseur
Projectontwikkelaar

Done

 Sociology={
'Education':'https://www.glassdoor.com/Job/netherlands-sociology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10009', 
"Human Resources and Staffing":'https://www.glassdoor.com/Job/netherlands-sociology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10026',
"Information Technology":"https://www.glassdoor.com/Job/netherlands-sociology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10013",
"Aerospace and Defense":"https://www.glassdoor.com/Job/netherlands-sociology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10002",
"Management and Consulting":"https://www.glassdoor.com/Job/netherlands-sociology-jobs-jobs-SRCH_IL.0,11_IN178_KO12,26.htm?industryNId=10006",
} 

rj
Policy advisor
Social researcher
Human resources specialist
Data analyst
Marketing research analyst
Project coordinator
Social media specialist
Event planner
Public relations specialist
Communications officer
Non-profit program coordinator
Diversity and inclusion officer
Social impact consultant
Community outreach specialist
Governmental affairs specialist
Health care coordinator
Youth development specialist
Training and development coordinator
Humanitarian aid worker
Crisis intervention specialist

Done

    Tandheelkunde={
'Education':'https://www.glassdoor.com/Job/netherlands--tandheelkunde-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10009', 
"Healthcare":'https://www.glassdoor.com/Job/netherlands--tandheelkunde-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10012',
"Information Technology":"https://www.glassdoor.com/Job/netherlands--tandheelkunde-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10013",
"Aerospace and Defense":"https://www.glassdoor.com/Job/netherlands--tandheelkunde-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10002",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands--tandheelkunde-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10026",
} 

rj
Tandarts
Orthodontist
Mondhygiënist
Kaakchirurg
Parodontoloog
Endodontoloog
Klinisch tandprotheticus
Tandtechnicus
Tandprotheticus
Implantoloog
Esthetisch tandarts
Kindertandarts
Gnatholoog
Orale patholoog
Forensisch odontoloog
Cariësconsulent
Preventie-assistent
Tandheelkundig adviseur
Tandheelkundig docent
Tandheelkundig onderzoeker

Done
Theaterwetenschap={
'Education':'https://www.glassdoor.com/Job/netherlands-theater-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10009', 
"Construction, Repair and Maintenance Services":'https://www.glassdoor.com/Job/netherlands-theater-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10007',
"Information Technology":"https://www.glassdoor.com/Job/netherlands-theater-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10013",
"Management and Consulting":"https://www.glassdoor.com/Job/netherlands-theater-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10006",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-theater-science-jobs-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10026",
} 

rj
Dramaturg
Theaterwetenschapper
Regisseur
Productieleider
Toneelmeester
Scenograaf
Acteur
Theatertechnicus
Lichtontwerper
Geluidsontwerper
Kostuumontwerper
Decorontwerper
Dramadocent
Theatercriticus
Programmamaker
Theatermarketingmedewerker
Zakelijk leider
Cultureel ondernemer
Festivalorganisator
Theatermaker


Done
    Duitse Taal en Cultuur={
'Education':'https://www.glassdoor.com/Job/netherlands--duitse-taal-en-cultuur-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10022', 
"Management and Consulting":'https://www.glassdoor.com/Job/netherlands--duitse-taal-en-cultuur-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10006',
"Information Technology":"https://www.glassdoor.com/Job/netherlands--duitse-taal-en-cultuur-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10013",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands--duitse-taal-en-cultuur-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10015",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands--duitse-taal-en-cultuur-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10026",
} 

rj
Vertaler Duits
Docent Duits
Duitse taaltrainer
Redacteur Duits
Tolken Duits
Cultureel adviseur Duitsland
Internationaal marketingmedewerker
Verkoper Duitstalig gebied
Duitslanddeskundige
Accountmanager Duitstalige klanten
Zakelijke correspondent Duits
PR-adviseur Duitsland
Content specialist Duitse taal
Vertegenwoordiger Duitse markt
Customer service medewerker Duitstalige klanten
Onderzoeker Duitse cultuur en geschiedenis
Duitse literatuurwetenschapper
Duitsland-analist
Webcare medewerker Duitstalige klanten
Duitslandjournalist

Literatuurwetenschap ={
'Education':'https://www.glassdoor.com/Job/netherlands-literary-studies-job-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10009', 
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-literary-studies-job-jobs-SRCH_IL.0,11_IN178_KO12,32.htm?industryNId=10026",
} 

rj
Content Writer
Editor
Copywriter
Proofreader
Literary Agent
Researcher
Librarian
Archivist
Communications Specialist
Public Relations Specialist
Translator
Blogger
Social Media Manager
Journalist
Scriptwriter
Screenwriter
Creative Writer
Grant Writer
Academic Advisor
Bookstore Manag

Done
 Taalwetenschap={
'Education':'https://www.glassdoor.com/Job/netherlands-linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10009', 
"Management and Consulting":'https://www.glassdoor.com/Job/netherlands-linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10006',
"Information Technology":"https://www.glassdoor.com/Job/netherlands-linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10013",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands-linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10015",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-linguistics-jobs-jobs-SRCH_IL.0,11_IN178_KO12,28.htm?industryNId=10026",
} 

rj
Linguistic Data Analyst
Computational Linguist
Language Teacher/Instructor
Lexicographer
Speech Language Pathologist
Translator/Interpreter
Foreign Language Teacher
Editor/Proofreader
Language Technologist
Speech Scientist
Sociolinguist
Semantics Analyst
Phonetics Researcher
Morphologist
Syntax Analyst
Discourse Analyst
Psycholinguist
Neurolinguist
Dialectologist
Language Documentation Specialist

Done
 Econometrie={
'Education':'https://www.glassdoor.com/Job/netherlands--econometrie-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10009', 
"Financial Services":'https://www.glassdoor.com/Job/netherlands--econometrie-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10010',
"Information Technology":"https://www.glassdoor.com/Job/netherlands--econometrie-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10013",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands--econometrie-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10015",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands--econometrie-jobs-jobs-SRCH_IL.0,11_IN178_KO12,29.htm?industryNId=10026",
} 

rj
Data Analyst
Risk Analyst
Quantitative Researcher
Business Analyst
Financial Analyst
Investment Analyst
Marketing Analyst
Econometrician
Statistician
Operations Research Analyst
Machine Learning Engineer
Actuary
Pricing Analyst
Product Analyst
Forecasting Analyst
Credit Risk Analyst
Economic Consultant
Supply Chain Analyst
Asset Management Analyst
Portfolio 

Done
    Artificial Intelligence={
'Education':'https://www.glassdoor.com/Job/netherlands-artificial-intelligence-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10009', 
"Financial Services":'https://www.glassdoor.com/Job/netherlands-artificial-intelligence-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10010',
"Information Technology":"https://www.glassdoor.com/Job/netherlands-artificial-intelligence-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10013",
"Healthcare":"https://www.glassdoor.com/Job/netherlands-artificial-intelligence-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10012",
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-artificial-intelligence-jobs-jobs-SRCH_IL.0,11_IN178_KO12,40.htm?industryNId=10026",
} 

rj

AI Researcher
Machine Learning Engineer
Data Scientist
Computer Vision Engineer
Natural Language Processing Engineer
Robotics Engineer
AI Architect
AI Consultant
AI Project Manager
Deep Learning Engineer
AI Programmer
Cognitive Computing Engineer
AI System Administrator
AI Marketing Analyst
AI Business Analyst
AI Chatbot Developer
AI Operations Manager
AI Security Analyst
AI Infrastructure Engineer
AI Product Manager                                                                                               
Aarde, Economie en Duurzaamheid={
'Education':'https://www.glassdoor.com/Job/netherlands--earth-sustainability-jobs-jobs-SRCH_IL.0,11_IN178_KO12,38.htm?industryNId=10009', 
"Energy, Mining and Utilities":'https://www.glassdoor.com/Job/netherlands--earth-sustainability-jobs-jobs-SRCH_IL.0,11_IN178_KO12,38.htm?industryNId=10019',
"Hotels and Travel Accommodation":"https://www.glassdoor.com/Job/netherlands--earth-sustainability-jobs-jobs-SRCH_IL.0,11_IN178_KO12,38.htm?industryNId=10025",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands--earth-sustainability-jobs-jobs-SRCH_IL.0,11_IN178_KO12,38.htm?industryNId=10015",
"Retail and Wholesale":"https://www.glassdoor.com/Job/netherlands--earth-sustainability-jobs-jobs-SRCH_IL.0,11_IN178_KO12,38.htm?industryNId=10022",
}                                                                                        rj                                                                          Sustainability Consultant
Environmental Policy Analyst
Renewable Energy Specialist
Green Building Architect
Sustainable Agriculture Manager
Carbon Footprint Analyst
Environmental Economist
Waste Reduction Coordinator
Water Resource Manager
Corporate Social Responsibility Coordinator
Climate Change Researcher
Natural Resource Manager
Eco-Tourism Specialist
Sustainable Transportation Planner
Sustainable Packaging Designer
Green Supply Chain Manager
Biodiversity Conservationist
Sustainable Business Manager
Community Outreach Coordinator
Sustainable Development Specialist                                                                              Aardewetenschappen={
'Education':'https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10009', 
"Aerospace & Defense":'https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10002',
"Human Resources and Staffing":"https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10026",
"Management and Consulting":"https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10006",
"Manufacturing":"https://www.glassdoor.com/Job/netherlands-earth-sciences-jobs-jobs-SRCH_IL.0,11_IN178_KO12,31.htm?industryNId=10015",
} 
rj                                                                                              Geologist
Hydrologist
Geophysicist
Seismologist
Environmental Scientist
Soil Scientist
Mineralogist
Paleontologist
Oceanographer
Geochemist
Petroleum Geologist
GIS Specialist
Geomorphologist
Volcanologist
Climate Scientist
Glaciologist
Planetary Scientist
Geotechnical Engineer
Remote Sensing Scientist
Geospatial Analyst


In [25]:
import pandas 
dev=pandas.read_csv('datas/datas_with_study/Cultuurwetenschappen_Education_last.csv')

In [29]:
dev.companies

0     The University of Texas Rio Grande Valley
1                                 Character Lab
2                                 Smith College
3                         University of Houston
4                             Aurora University
5                           University of Maine
6                        University of Kentucky
7                        University of Kentucky
8                           New York University
9                          Creighton University
10                      University of Minnesota
11                    University of New England
12                   Indian River State College
13                   Glendale Community College
14                University of Texas at Austin
15                               William & Mary
16                        University of Florida
17          University of Massachusetts Amherst
18                       University of Maryland
19                  California State University
20      Contra Costa Community College D